In [4]:
import importlib
import smoke_detection
importlib.reload(smoke_detection)
from smoke_detection import detect_smoke
import vehicle_detection
importlib.reload(vehicle_detection)

<module 'vehicle_detection' from 'c:\\Users\\Abad\\Desktop\\FYP\\yolo_test\\combine_2\\final_realtime\\vehicle_detection.py'>

In [7]:
import os
import cv2
import time
import csv
from collections import Counter
from ultralytics import YOLO
import easyocr

# Import existing functions and counters
from license_plate_detection import process_license_plate_frame, license_plate_counts
from smoke_detection import detect_smoke  # assume this function works on any input image (cropped or full)
from vehicle_detection import detect_vehicle

# --- Paths for Output ---
output_dir = r"./output"
os.makedirs(output_dir, exist_ok=True)
csv_file_path = os.path.join(output_dir, "detected_plates.csv")

# --- Track plates saved this session ---
saved_plates_session = set()

def save_plate_csv(plate_text, extra_info="SmokingVehicle"):
    if plate_text in saved_plates_session:
        return

    saved_plates_session.add(plate_text)
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
    row = [timestamp, plate_text, extra_info]

    file_exists = os.path.isfile(csv_file_path)
    with open(csv_file_path, mode="a", newline="", encoding="utf-8") as csv_file:
        writer = csv.writer(csv_file)
        if not file_exists:
            writer.writerow(["DateTime", "LicensePlate", "ExtraInfo"])
        writer.writerow(row)
    print(f"Saved new license plate to CSV: {plate_text}")

def process_frame_strategy(frame):
    """
    Process a frame using a step-by-step strategy:
      1. Detect vehicles.
      2. For each vehicle, crop the region and run smoke detection.
      3. If smoke is confirmed, run license plate detection on that vehicle region.
      4. Annotate the full frame with the "smoking vehicle" info and save the license plate to CSV.
    
    Returns:
        Annotated frame.
    """
    output_frame = frame.copy()

    # Detect vehicles in the full frame
    vehicle_detections = detect_vehicle(frame)
    
    # Iterate over each detected vehicle
    for det in vehicle_detections:
        box = det["box"]  # [x1, y1, x2, y2]
        # Crop the region corresponding to the vehicle
        x1, y1, x2, y2 = box
        vehicle_roi = frame[y1:y2, x1:x2]
        
        # Safety check: if the ROI is empty, skip
        if vehicle_roi.size == 0:
            continue
        
        # Run smoke detection on the cropped vehicle ROI.
        # (Assuming detect_smoke can be run on any image; if it returns (smoke_detected, processed_image),
        # adapt accordingly. Here we assume it returns a boolean and a processed image.)
        smoke_detected, _ = detect_smoke(vehicle_roi)
        
        # If the vehicle region shows smoke, run license plate detection on the same region.
        if smoke_detected:
            # Option 1: Run license plate detection on the vehicle ROI.
            lp_roi = process_license_plate_frame(vehicle_roi.copy())
            # Optionally, if process_license_plate_frame returns an annotated image, you could extract
            # the license plate string from the global license_plate_counts (if that's how your module works)
            if license_plate_counts:
                most_common_plate = license_plate_counts.most_common(1)[0][0]
                # Save the license plate to CSV (only once per session)
                save_plate_csv(most_common_plate)
                # Annotate the full frame: Draw the bounding box in red and label it.
                cv2.rectangle(output_frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                cv2.putText(output_frame, f"vehicle (smoke) - Plate: {most_common_plate}", 
                            (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            else:
                # If no plate is detected, at least label the vehicle as smoking.
                cv2.rectangle(output_frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                cv2.putText(output_frame, "vehicle (smoke)", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        else:
            # For vehicles without smoke, you might want to annotate them differently (optional)
            cv2.rectangle(output_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(output_frame, "vehicle", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    return output_frame

def run_detection(source=0):
    """
    Runs the combined strategy on either:
      - Webcam (source=0)
      - Video file (source="video.mp4")
    Saves the resulting video to ./output/predicted_output.avi
    """
    # Clear global counters and session data
    license_plate_counts.clear()
    saved_plates_session.clear()

    cap = cv2.VideoCapture(source)
    if not cap.isOpened():
        print(f"Error: Could not open source: {source}")
        return

    # --- Setup VideoWriter ---
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps == 0 or fps is None:
        fps = 25

# Find next available filename
    i = 1
    while True:
        output_path = os.path.join(output_dir, f"predicted_output{i}.avi")
        if not os.path.exists(output_path):
            break
        i += 1

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    print(f"Saving output video to: {output_path}")


    while True:
        ret, frame = cap.read()
        if not ret:
            print("End of video or error reading frame.")
            break

        processed_frame = process_frame_strategy(frame)
        out.write(processed_frame)
        cv2.imshow("Vehicle with Confirmed Smoke Detection", processed_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()


def main():
    # Use 0 for real-time camera, or provide a path like 'input_video.mp4'
    # Example usage:
    # run_detection(0)  # real-time
    run_detection("input/4.mp4")  # video file

if __name__ == "__main__":
    main()


Saving output video to: ./output\predicted_output2.avi

0: 640x384 1 truck, 99.4ms
Speed: 6.2ms preprocess, 99.4ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x608 (no detections), 70.0ms
Speed: 1.6ms preprocess, 70.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 608)

0: 640x384 1 truck, 107.7ms
Speed: 2.3ms preprocess, 107.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x608 (no detections), 66.4ms
Speed: 13.1ms preprocess, 66.4ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 608)

0: 640x384 1 truck, 116.8ms
Speed: 0.0ms preprocess, 116.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x608 (no detections), 66.7ms
Speed: 0.0ms preprocess, 66.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 608)

0: 640x384 1 truck, 99.9ms
Speed: 2.0ms preprocess, 99.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x608 (no detections), 77.1ms
Speed